# 어노테이션 데이터 사용하기

**[목차]**
* **데이터 전처리 :** 어노테이션 데이터를 하나의 데이터프레임으로 변환하기
* **데이터 분석 및 시각화 :** 어노테이션 데이터를 이미지 위에 출력하기
* **다양한 카메라 이미지에 반영해보기**


## 1. 데이터 전처리 : 어노테이션 데이터를 하나의 데이터프레임으로 변환하기

이번 실습에서는 어노테이션 데이터를 이용하는 실습을 진행

이를 위해 지난 시간에서 사용한 `json_to_dataframe` 함수를 이용하여 JSON 파일을 "메타데이터"와 "어노테이션 데이터" 로 나누고 각각 데이터프레임으로 저장 

In [ ]:
import pandas as pd
import glob
import json

In [ ]:
def json_to_dataframe(file_path):

    # 주어진 파일 경로에서 JSON 파일을 열고 데이터를 읽어옴
    with open(file_path, 'r') as file:
        data = json.load(file)


    # annotations를 제외한 모든 데이터를 포함하는 데이터프레임 생성
    df = pd.json_normalize(data)

    
    # 'annotations'라는 열이 df의 열 목록에 있는지 확인
    if 'annotations' in df.columns:
        
        # 'annotations' 열이 존재하면, 데이터를 'annotations' 열에서 추출하여 JSON 형식으로 펼침
        annotations_df = pd.json_normalize(data, 'annotations', sep='_', errors='ignore')

        # 'annotations' 열을 원래 DataFrame(df)에서 제거
        df.drop(columns=['annotations'], inplace=True)
    
    else:
        # 'annotations' 열이 존재하지 않는 경우, annotations_df를 None으로 설정
        annotations_df = None

    
    return df, annotations_df

### 1.1 하나의 JSON 파일의 어노테이션 데이터 사용하기

자율주행 자동차의 시스템마다 다르지만 현재 타겟이 되는 자율주행 자동차의 시나리오는 다음과 같이 구성되어 있음

이번 실습은 하나의 JSON 파일의 어노테이션 데이터를 이용해서 이를 이미지 위에 도형을 그리는 실습을 진행

In [ ]:
annotation_path = "./data/label_2dbb/FR-View-CMR-Wide/1656041450209_FR-View-CMR-Wide_2DBB_GT.json"

meta_df, annotations_df = json_to_dataframe(annotation_path)

annotations_df

본격적으로 분석하기 전, 메타데이터(meta_df)에는 짝꿍이 되는 이미지 파일의 경로를 포함하고 있음

In [ ]:
meta_df

- **parent_path :** 이미지 파일의 경로 
- **filename :**  이미지 파일의 이름 (확장자 미포함)
- **file_format :** 이미지 파일의 확장자 


현재의 실습을 구성하기 위해 폴더 구조, 폴더 이름을 변경해서 JSON 파일에 저장되어 있는 경로(path)가 정확히 일치하지 않는 문제가 있음

그래서 `parent_path`를 간단히 사용할 수 있도록 전처리 과정을 진행

`split`과 `join`을 이용하여 불필요한 데이터는 제거

In [ ]:
meta_df["parent_path"].values[0]

In [ ]:
meta_df["parent_path"].values[0].split('/')

In [ ]:
"/".join(meta_df["parent_path"].values[0].split('/')[2:])

In [ ]:
meta_df["filename"].values[0]

In [ ]:
meta_df["file_format"].values[0]

메타데이터에서 추출한 `parent_path`, `filename`, `file_format` 을 이용하면 이미지 파일의 경로를 특정할 수 있음

In [ ]:
folder_path = "/".join(meta_df["parent_path"].values[0].split('/')[2:]) 
file_name = meta_df["filename"].values[0]
file_format = meta_df["file_format"].values[0]


img_path = "data/" + folder_path + "/" + meta_df["filename"].values[0] + "."+ meta_df["file_format"].values[0]
img_path

`os` 라이브러리의 `path.join`을 이용하는 방법도 존재

In [ ]:
import os

image_path = os.path.join("data", folder_path, (file_name + "." + file_format))
image_path

## 2. 데이터 분석 및 시각화 : 어노테이션 데이터를 이미지 위에 출력하기

### 2.1 source_img의 이미지 출력하기 

추출된 이미지 경로(image_path)를 이용하여 어떤 이미지인지 확인하는 작업 진행

이를 위해 **두 가지 라이브러리**를 사용 

1. **matplotlib 라이브러리 :** 데이터를 시각화하거나 이미지를 출력하기 위한 다양한 기능 제공
2. **Pillow 라이브러리 :** 다양한 이미지 파일 형식(png, jpeg, gif 등)을 지원하며 이미지를 변환, 분석 등 다양한 기능 제공


`open`을 이용하면 이미지 경로의 파일을 출력할 수 있음

In [ ]:
import matplotlib.pyplot as plt  
from PIL import Image 

image = Image.open(image_path)  

# 이미지 표시  
plt.imshow(image)  
plt.title(file_name + "." + file_format)  # 이미지 파일 이름을 제목으로 표시  

plt.show()

### 2.2 바운딩 박스 그리기

어노테이션 데이터(annotations_df) 를 확인하면 다양한 데이터들이 있음

그 중에서 `points`  주목해야 함

In [ ]:
annotations_df.head()

가장 첫번째 행의 points 데이터를 확인하면 다음과 같은 특징이 있음

1. 2개의 숫자를 가진 리스트가 있음
2. 1번의 리스트가 총 4개 있음

아래의 코드에서 숫자 0을 다른 숫자로 변경해도 똑같은 형태인 것을 확인할 수 있음

In [ ]:
annotations_df.iloc[0]["points"]

이 데이터는 이미지 위의 좌표를 의미

보통 이미지 좌표는 네 꼭지점을 기준으로 우측 상단을 (0, 0)을 기준으로 함

`points` 좌표는 각각 P1, P2, P3, P4를 의미

좌표들을 이용해서 이미지 위에 도형을 그릴려면 `matplotlib`의 `Polygon` 기능을 사용
* `closed` : 다각형이 닫힌 형태임을 지정
* `linewidth` : 다각형 테두리의 두께를 설정
* `edgecolor` : 다각형 테두리의 색상을 설정
* `facecolor` : 다각형 내부의 색상을 설정 ('none' = 투명)

In [ ]:
from matplotlib.patches import Polygon

image = Image.open(image_path)

fig, ax = plt.subplots()
ax.imshow(image)  
ax.set_title(file_name + "." + file_format)  

polygon = Polygon(
    annotations_df.iloc[0]["points"], 
    closed=True, 
    linewidth=2, 
    edgecolor='r', 
    facecolor='none'
)  
ax.add_patch(polygon)

plt.show()

### 2.3 바운딩 박스 여러개 그리기 

하지만 좌표 데이터는 하나가 아니라 아주 많음

현재의 데이터를 확인하면 13개의 좌표가 존재

In [ ]:
for point in annotations_df["points"] : 
    print(point)

반복문을 사용하면 쉽게 이미지 위에 도형을 그릴 수 있음

즉, 어노테이션 데이터의 객체마다 **바운딩 박스(Bounding Box)** 를 그릴 수 있음

In [ ]:
image = Image.open(image_path)

fig, ax = plt.subplots()
ax.imshow(image)  
ax.set_title(file_name + "." + file_format)

for point in annotations_df["points"] : 
    polygon = Polygon(point, closed=True, linewidth=2, edgecolor='r', facecolor='none')  
    ax.add_patch(polygon)

plt.show()

### 2.4 객체 별로 다양한 색상의 바운딩 박스 그리기
하지만 출력된 이미지는 전부 한 색상으로 된 바운딩 박스들이어서 어떤 객체인지 구분이 어려움

어노테이션 데이터(annotations_df)를 확인하면 어떤 객체인지 알려주는 `label`을 확인할 수 있음

In [ ]:
annotations_df.head()

In [ ]:
annotations_df["label"].unique()

하지만 `vehicle` 과 `traffic_sign`이 내포하는 범위가 상당히 넓음

`attribute` 컬럼에서 이보다 더 상세한 의미를 확인할 수 있음

In [ ]:
annotations_df["attribute"].unique()

앞에서 살펴본 좌표 데이터(`points`)와 어트리뷰트(`attribute`) 데이터를 이용하여 더욱 뚜렷하게 바운딩 박스를 그리는 작업을 진행

In [ ]:
annotations_df[["points", "attribute"]]

Pandas 라이브러리의 `iterrows` 또는 `itertuples` 메소드를 사용하면 for문 처럼 **반복(iterate) 처리**를 할 수 있음

대신, 사용법이 for 문과 조금 다르기 때문에 주의 필요

#### (1) dataframe.iterrows 를 이용한 방법

In [ ]:
dict_1 = {
    'col1': [4, 1, 5, 3, 2],
    'col2': [6, 7, 8, 9, 10],
    'col3': [11, 12, 13, 14, 15],
    'col4': [16, 17, 18, 19, 20]
}

df_1 = pd.DataFrame(dict_1)
df_1

In [ ]:
for row in df_1.iterrows():
    print(row)
    print(type(row))
    
    print()

In [ ]:
for idx, row in df_1.iterrows():
    print(idx)
    print(type(idx))

    print(row)
    print(type(row))
    
    print()

In [ ]:
for idx, annotaion in annotations_df[["points", "attribute"]].iterrows() :
    point = annotaion["points"]
    attribute = annotaion["attribute"]
    
    print(point)
    print(attribute)
    print()

#### (2) dataframe.itertuples 를 이용한 방법

In [ ]:
df_1

In [ ]:
for row in df_1.itertuples():
    print(row)
    print(type(row))
    
    print()

In [ ]:
for row in df_1.itertuples():
    print(row.col2)

In [ ]:
for annotaion in annotations_df[["points", "attribute"]].itertuples() : 
    point = annotaion.points
    attribute = annotaion.attribute
    
    print(point)
    print(attribute)
    print()

`iterrows` 또는 `itertuples` 메소드를 사용하여 `attribute` 별로 색상을 다르게 할 수 있음
* car = 빨강
* adult = 초록
* truck_s = 파랑
* motorcycle = 분홍
* 그 외 = 주황

In [ ]:
image = Image.open(image_path)

fig, ax = plt.subplots()
ax.imshow(image)  
ax.set_title(file_name + "." + file_format)

for annotaion in annotations_df[["points", "attribute"]].itertuples() : 
    point = annotaion.points
    attribute = annotaion.attribute
    
    if attribute == "car" : 
        polygon = Polygon(point, closed=True, linewidth=1, edgecolor='red', facecolor='none')  

    elif attribute == "adult" :
        polygon = Polygon(point, closed=True, linewidth=1, edgecolor='green', facecolor='none')  
    
    elif attribute == "truck_s" :
        polygon = Polygon(point, closed=True, linewidth=1, edgecolor='blue', facecolor='none')  
        
    elif attribute == "motorcycle" :
        polygon = Polygon(point, closed=True, linewidth=1, edgecolor='pink', facecolor='none')  

    else :
        polygon = Polygon(point, closed=True, linewidth=1, edgecolor='orange', facecolor='none')  

    
    ax.add_patch(polygon)

plt.show()

위의 코드를 조금 더 효율적으로 재구성할 수 있음

`get` 메소드를 사용하면 딕셔너리에서 주어진 키(key)에 해당하는 값을 반환

In [ ]:
color_map = {  
    "car": "red",  
    "adult": "green",  
    "truck_s": "blue",  
    "motorcycle": "pink"  
}  
default_color = "orange" 

for annotation in annotations_df.itertuples():  
    points = annotation.points  
    attribute = annotation.attribute  

    color = color_map.get(attribute, default_color)  

    print(attribute, ":", color)

이 점을 이용해서 위의 코드를 다시 작성하면 다음과 같이 작성할 수 있음

In [ ]:
# 이미지 불러오기  
image = Image.open(image_path)  

# 플롯 생성  
fig, ax = plt.subplots()  
ax.imshow(image)  
ax.set_title(f"{file_name}.{file_format}")  

# 속성별 색상 매핑  
color_map = {  
    "car": "red",  
    "adult": "green",  
    "truck_s": "blue",  
    "motorcycle": "pink"  
}  
default_color = "orange"  

# 각 주석에 대해 다각형 그리기  
for annotation in annotations_df.itertuples():  
    points = annotation.points  
    attribute = annotation.attribute  

    # 색상 매핑 및 다각형 생성  
    color = color_map.get(attribute, default_color)  
    polygon = Polygon(points, closed=True, linewidth=1, edgecolor=color, facecolor='none')  

    # 다각형을 플롯에 추가  
    ax.add_patch(polygon)  

# 이미지 표시  
plt.show()

해당 코드를 함수로 만들면 간단하게 바운딩박스 작업 가능

In [ ]:
def img_2d_bounding_box(image_path) : 

    file_name = image_path.split("/")[-1]
    
    # 이미지 불러오기  
    image = Image.open(image_path)  

    # 플롯 생성  
    fig, ax = plt.subplots()  
    ax.imshow(image)
    ax.set_title(file_name)  

    # 속성별 색상 매핑  
    color_map = {  
        "car": "red",  
        "adult": "green",  
        "truck_s": "blue",  
        "motorcycle": "pink"  
    }  
    default_color = "orange"  

    # 각 주석에 대해 다각형 그리기  
    for annotation in annotations_df.itertuples():  
        points = annotation.points  
        attribute = annotation.attribute  

        # 색상 매핑 및 다각형 생성  
        color = color_map.get(attribute, default_color)  
        polygon = Polygon(points, closed=True, linewidth=1, edgecolor=color, facecolor='none')  

        # 다각형을 플롯에 추가  
        ax.add_patch(polygon)  

    # 이미지 표시  
    plt.show()

In [ ]:
img_2d_bounding_box(image_path)

## 3. 다양한 폴더에 반영해보기 

현재의 자율주행 차량은 4개의 카메라를 탑재하고 있어서 `label_2dbb`, `source_image` 폴더에는 4개의 폴더가 존재

* **FR-View-CMR-Wide :** 전방 카메라 폴더 
* **RR-Left-View-CMR-Narrow :** 좌측 후방 카메라 폴더
* **RR-Right-View-CMR-Narrow :** 우측 후방 카메라 폴더
* **RR-View-CMR-Wide :** 후방 카메라 폴더 

`glob` 라이브러리를 사용하여 지정된 패턴에 일치하는 파일 경로의 리스트를 생성

In [ ]:
folder_list = [
    "FR-View-CMR-Wide", 
    "RR-Left-View-CMR-Narrow", 
    "RR-Right-View-CMR-Narrow", 
    "RR-View-CMR-Wide"
    ]

path = f"data/label_2dbb/{folder_list[0]}/*_{folder_list[0]}_2DBB_GT.json"

file_paths = glob.glob(path)
file_paths

해당 리스트를 이용하여 이미지의 경로를 추출하는 작업 진행

In [ ]:
for file_path in file_paths : 
    meta_df, annotations_df = json_to_dataframe(annotation_path)

    folder_path = "/".join(meta_df["parent_path"].values[0].split('/')[2:]) 
    file_name = meta_df["filename"].values[0]
    file_format = meta_df["file_format"].values[0]

    image_path = os.path.join("data", folder_path, (file_name + "." + file_format))

    print(image_path)

사전에 정의한 `json_to_dataframe`함수를 이용하여 어노테이션 데이터를 추출하고 `img_2d_bounding_box` 함수를 이용하여 바운딩박스 작업을 진행

FR-View-CMR-Wide 폴더에 있는 이미지들을 어노테이션 데이터에 맞게 전부 바운딩박스가 그려진 것을 확인할 수 있음

In [ ]:
for file_path in file_paths : 
    meta_df, annotations_df = json_to_dataframe(annotation_path)

    folder_path = "/".join(meta_df["parent_path"].values[0].split('/')[2:]) 
    file_name = meta_df["filename"].values[0]
    file_format = meta_df["file_format"].values[0]

    image_path = os.path.join("data", folder_path, (file_name + "." + file_format))

    img_2d_bounding_box(image_path)

### [TODO] 어노테이션 데이터를 이용하여 다른 카메라 이미지에 바운딩 박스 그리기

label_2dbb, source_image 폴더에는 다른 카메리의 주행 데이터가 존재합니다. 

위에서 작성한 코드들을 이용하여 주행 이미지에 바운딩 박스를 그리는 코드를 작성하세요.

*※ 본 실습은 제출이 없는 실습입니다.*

In [ ]:
None